
https://registry.opendata.aws/nyc-tlc-trip-records-pds/

In [ ]:
sc

In [56]:
from pyspark.sql import SQLContext
from pyspark.sql import functions as f

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sql_context = SQLContext(sc)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df1 = sql_context.read.parquet("s3://nyc-tlc/trip data/fhv*2017*",)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df2 = sql_context.read.parquet("s3://nyc-tlc/trip data/fhv*2018*",)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df = df1.union(df2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

453184311

In [13]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00001|2017-06-01 00:04:00|2017-06-07 01:55:36|        null|        null|   null|                      |
|              B00001|2017-06-01 00:47:00|2017-06-09 01:25:40|        null|        null|   null|                      |
|              B00001|2017-06-01 00:07:00|2017-06-15 00:52:17|        null|        null|   null|                      |
|              B00001|2017-06-01 00:30:00|2017-06-03 02:18:52|        null|        null|   null|                      |
|              B00001|2017-06-01 00:45:00|2017-06-10 01:49:42|        null|        null|   null|                      |
|              B00001|2017-06-01 00:25:0

In [14]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: double (nullable = true)
 |-- DOlocationID: double (nullable = true)
 |-- SR_Flag: double (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)

In [17]:
df_zones = sql_context.read.csv("s3://nyc-tlc/misc/taxi _zone_lookup.csv", header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_zones.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [21]:
df_zones.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)

In [32]:
df3 = df.join(df_zones, df.PUlocationID==df_zones.LocationID, how="inner").\
select("pickup_datetime","dropOff_datetime","DOlocationID", df_zones.Zone.alias("PUZone"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df3.count()#(left)453184311 #(inner)373840936

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

373840936

In [37]:
df4 = df3.join(df_zones, df3.DOlocationID==df_zones.LocationID, how="inner").\
select("pickup_datetime","dropOff_datetime","PUZone", df_zones.Zone.alias("DOZone"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df4.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropOff_datetime|      PUZone|      DOZone|
+-------------------+-------------------+------------+------------+
|2017-06-01 00:00:15|2017-06-01 00:05:42|North Corona|North Corona|
|2017-06-01 00:00:54|2017-06-01 00:07:00|      Corona|      Corona|
|2017-06-01 00:45:37|2017-06-01 01:01:52|    Woodside|    Elmhurst|
|2017-06-01 00:41:31|2017-06-01 01:27:57|   Rego Park|      Corona|
|2017-06-01 00:23:36|2017-06-01 00:34:24|      Corona|      Corona|
+-------------------+-------------------+------------+------------+
only showing top 5 rows

In [40]:
df4.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUZone: string (nullable = true)
 |-- DOZone: string (nullable = true)

In [49]:
df5 = df4.select("dropOff_datetime","pickup_datetime","PUZone","DOZone",((df4.dropOff_datetime.cast("long")-df4.pickup_datetime.cast("long"))/60).alias("time"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
df5.orderBy(df5.time.desc()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+----------------+--------------+---------+
|   dropOff_datetime|    pickup_datetime|          PUZone|        DOZone|     time|
+-------------------+-------------------+----------------+--------------+---------+
|2019-11-29 22:29:00|2017-11-29 20:59:00|  Newark Airport|            NA|1051290.0|
|2019-09-21 01:50:00|2018-09-20 00:30:00|              NV|            NV| 527120.0|
|2019-09-21 02:30:00|2018-09-20 01:45:00|              NV|            NV| 527085.0|
|2019-09-20 17:15:00|2018-09-20 13:45:00|              NV|            NV| 525810.0|
|2019-09-20 18:15:00|2018-09-20 15:15:00|              NV|            NV| 525780.0|
|2019-09-20 19:30:00|2018-09-20 16:30:00|              NV|            NV| 525780.0|
|2019-09-20 18:00:00|2018-09-20 15:00:00|              NV|            NV| 525780.0|
|2019-09-20 18:30:00|2018-09-20 15:30:00|              NV|            NV| 525780.0|
|2019-09-20 19:30:00|2018-09-20 16:30:00|              NV|            NV| 52

In [64]:
# tiempo promedio viaje
#Saint Albans|Bushwick South
df5.where("PUZone == 'Saint Albans' and DOZone == 'Bushwick South'").select(f.avg("time")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|        avg(time)|
+-----------------+
|307.3221455351057|
+-----------------+